In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py


from holodeck import plot, detstats
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC, GYR, PC
import holodeck as holo
from holodeck.sams import sam

import hasasia.sim as hsim

import sys
sys.path.append('/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation')
import anatomy as anat

# Try again

In [ ]:
sam = holo.sams.Semi_Analytic_Model(shape=10)

# Nothing below this makes sense

# Get PSpace Info

In [ ]:
# use one file to get the shape
npz = np.load('/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation/anatomy_uniform09A_fullshape/hard_time_p0.5_0.5_0.5_0.5_0.5_0.5_s91_81_101.npz',
                        allow_pickle=True)             
print(npz.files)
data = npz['data']
params = npz['params']
hard_name = npz['hard_name']
shape = npz['shape']
target_param = npz['target_param']

npz.close()

# get param names
pspace = holo.param_spaces.PS_Uniform_09A(holo.log, nsamples=1, sam_shape=shape, seed=None)
param_names = pspace.param_names
print(param_names)
print(f"{shape=}")
print(f"{data[0]['hc_ss'].shape}")
print(f"{data[0].keys()=}")

# # set directory path
# sam_loc = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation/anatomy_uniform09A_fullshape/'
# save_dir=sam_loc+'/figures'       

In [ ]:
print(data[0]['gwb_params'][0].shape)

### Make Model

In [ ]:
sam1, hard1 = pspace.model_for_params(params[1])
fobs_gw_cents = data[1]['fobs_cents']
fobs_gw_edges = data[1]['fobs_edges']
NFREQS, NREALS, NLOUDEST = [*data[0]['hc_ss'].shape]
print(f"{NFREQS=}, {NREALS=}, {NLOUDEST=}")

In [ ]:
fobs_orb_cents = fobs_gw_cents/2
edges, dnum, redz_final, dets = sam1._dynamic_binary_number_at_fobs_consistent(hard1, fobs_orb_cents, details=True)

In [ ]:
dadt = dets['dadt']
sepa = dets['sepa']
tau = dets['tau']

### Timescales

In [ ]:
# def tau_from_dadt(dadt, sepa):
#     """ tau = dt/dlna = dt/(da/a) = a*(dt/da) = a/(da/dt)"""
#     tau = (sepa)/dadt
#     return tau

# Plot Hardening Time vs. Separation

In [ ]:
print(sings.par_names)

In [ ]:
print(f"{sepa.shape=}, {tau.shape=}")
print(holo.utils.stats(sepa))
print(holo.utils.stats(tau))

In [ ]:
mm=-1
qq=-1
zz=-1
for mm in [5,50,80]:
    for qq in [5,50, 80]:
        for zz in [5,50,80]:
            plt.plot(fobs_gw_cents, sepa[mm,qq,zz])

In [ ]:
print(sepa.shape)

In [ ]:
xlabels = ['Binary Separation [pc]', 'GW Frequency [nHz]']
ylabels = ['Hardening Time [Gyr]', 'GW Characteristic Strain']

fig, axs = plot.figax(nrows=2, figsize=(5,6)
                     )
for ii,ax in enumerate(axs):
    ax.set_xlabel(xlabels[ii])
    ax.set_ylabel(ylabels[ii])

fig.tight_layout()

x1 = sepa/PC
y1 = dets['tau']/GYR

for mm in np.arange(0, 90, 10):
    for qq in np.arange(0,80,10):
        for zz in np.arange(0,100,10):
            axs[0].plot(x1[mm,qq,zz], y1[mm,qq,zz], alpha=0.5)
# axs[0].set_xlim(10**3, 10**-3)


# x2_bg = fobs_gw_cents





# Let's just copy Luke's notebook

In [ ]:
import tqdm

In [ ]:
NSTEPS = 10

mtot_range = [3e8*MSOL, 3e9*MSOL]
mtot_hirng = [3e9*MSOL, 3e10*MSOL]
mrat_range = [0.2, 1.0]
redz_range = [0, np.inf]

space = pspace # from above

# hard_time binary lifetimes 
times_list =  [params[0]['hard_time'], params[1]['hard_time'], params[2]['hard_time']]
# hard_gamma_inner power law indices
inner_list = [params[1]['hard_gamma_inner'], ]
# range of binary separations to plot
sepa = np.logspace(-3, 3, NSTEPS)[::-1] * PC

time_hcss = []
time_hcbg = []
time_taus = []
time_taus_high = []

# Iterate over target lifetimes
for tt, time in tqdm.tqdm(enumerate(times_list)):
    _hcss = []
    _hcbg = []
    _taus = []
    _taus_high = []

    # iterate over outer power-law indices
    for inner in tqdm.tqdm(inner_list):
        # set custom parameters:
        # using my parameters from above
        params_step = params[1] # midpoints
        params_step['hard_time'] = time
        params_step['hard_gamma_inner'] = inner
        sam, hard = pspace.model_for_params(params_step)

        # calculate hc_bg and hc_ss at bin centers, between the given bin edges
        _hcbg_step, _hcss_step, = sam.gwb(fobs_gw_edges, hard, 
                                    loudest = NLOUDEST, realize=NREALS)
        _hcss.append(_hcss_step)
        _hcbg.append(_hcbg_step)
    
        # _hcss.append(data[tt]['hc_ss'])
        # _hcbg.append(data[tt]['hc_bg'])
 
        # calculate binary properties at target separations
        _edges, _dnum, _redz_final, _details = sam._dynamic_binary_number_at_sepa_consistent(
            hard, sepa, details=True) # it would be better if I saved these details when I first calculated them!
        
        # select the bins with target binary parameters
        # I could update this to select out my single source bins
        sel_mtot = (mtot_range[0] < sam.mtot) & (sam.mtot <= mtot_range[1])
        sel_himt = (mtot_hirng[0] < sam.mtot) & (sam.mtot <= mtot_range[1])
        sel_mrat = (mrat_range[0] < sam.mrat) & (sam.mrat <= mrat_range[1])
        sel_redz = (redz_range[0] < sam.redz) & (sam.redz <= redz_range[1])
        sel = (
            sel_mtot[:, np.newaxis, np.newaxis] *
            sel_mrat[np.newaxis, :, np.newaxis] * 
            sel_redz[np.newaxis, np.newaxis, :]
        )

        sel_high = (
            sel_himt[:, np.newaxis, np.newaxis] *
            sel_mrat[np.newaxis, :, np.newaxis] * 
            sel_redz[np.newaxis, np.newaxis, :]
        )

        tau = _details['tau'][sel].T
        tau_high = _details['tau'][sel_high].T
        _taus.append(tau)
        _taus_high.append(tau_high)

    time_hcss.append(_hcss)
    time_hcbg.append(_hcbg)
    time_taus.append(_taus)
    time_taus_high.append(_taus_high)
        

# Plot Results

In [ ]:
fig, axs = plot.figax_single(height=7, nrows=2,  hspace=0.35, bottom=0.1)

xx = sepa/PC
YR_LABEL_PAD = -4

# ------------------------    1    ----------------------------
ax = axs[0]
ax.set(xlabel=plot.LABEL_SEPARATION_PC, ylabel=plot.LABEL_HARDENING_TIME, xscale='log', yscale='log')
ax.invert_xaxis()

ax.axhline(times_list[1], color='k', alpha=0.65)
ax.axhline(times_list[0], color='k', ls='--', alpha=0.25)


gamma_labels = []
gamma_handles = []
time_labels = []
time_handles = []
colors = []
inner_idx = 1
for ii, tau in enumerate(time_taus[:,inner_idx]):
    yy = tau / GYR
    
    hh = plot.draw_med_conf(ax, xx, yy, fracs=[0.5], filter=True)
    colors.append(hh[0].get_color())
    time_handles.append(hh[0])
    time_labels.append(f"${times_list[ii]:.1f}$")
    if ii == 0:
        gamma_labels.append(f"${inner_list[inner_idx]:+.1f}$")
        gamma_handles.append(hh)

# time_idx = 0        
# for ii, tau in enumerate(inner_taus[time_idx]):
#     yy = tau / GYR
#     yy = [np.median(y[y>0.0]) for y in yy]
#     hh, = ax.plot(xx, yy, color=colors[ii], ls='--', alpha=0.35)    
#     if ii == 0:
#         time_handles.append(hh)
#         time_labels.append(f"${times_list[time_idx]:.1f}$")

# leg = ax.legend(gamma_handles, gamma_labels, loc='lower left', 
#                 ncol=len(gamma_handles), title='$\\nu_\mathrm{inner}$', title_fontsize=14)

Look at sspar range

In [ ]:
print(sspar[0].shape)

In [ ]:
sspar = data[1]['sspar']
print(holo.utils.stats(sspar[0][...,-1]/MSOL))
print('min: %e, max:%e' % (np.min(sspar[0][...,-1]) / MSOL, np.max(sspar[0][...,-1])/ MSOL))

In [ ]:
fig, ax = plot.figax()
ax.hist(sspar[0].flatten()/MSOL, color='b', alpha=0.5, label='5 loudest',
         bins=100, log=True)
ax.hist(sspar[0][...,-1].flatten()/MSOL, color='r', alpha=0.5, label='1 loudest', 
        bins=100, log=True)

# Varying gamma_inner

In [ ]:
NSTEPS = 10

mtot_range = [3e8*MSOL, 3e9*MSOL]
mtot_hirng = [3e9*MSOL, 3e10*MSOL]
mrat_range = [0.2, 1.0]
redz_range = [0, np.inf]

space = pspace # from above

# hard_time binary lifetimes 
times_list = [params[1]['hard_time'],]]
# hard_gamma_inner power law indices
inner_list = [params[0]['hard_gamma_inner'], params[1]['hard_gamma_inner'], params[2]['hard_gamma_inner']]
# range of binary separations to plot
sepa = np.logspace(-3, 3, NSTEPS)[::-1] * PC

inner_hcss = []
inner_hcbg = []
inner_taus = []
inner_taus_high = []

# Iterate over target lifetimes
for time in tqdm.tqdm(times_list):
    _hcss = []
    _hcbg = []
    _taus = []
    _taus_high = []

    # iterate over outer power-law indices
    for inner in tqdm.tqdm(inner_list):
        # set custom parameters:
        # using my parameters from above
        params_step = params[1] # midpoints
        params_step['hard_time'] = time
        params_step['hard_gamma_inner'] = inner
        sam, hard = pspace.model_for_params(params_step)

        # calculate hc_bg and hc_ss at bin centers, between the given bin edges
        _hcbg_step, _hcss_step, = sam.gwb(fobs_gw_edges, hard, 
                                    loudest = NLOUDEST, realize=NREALS)
        _hcss.append(_hcss_step)
        _hcbg.append(_hcbg_step)
 
        # calculate binary properties at target separations
        _edges, _dnum, _redz_final, _details = sam._dynamic_binary_number_at_sepa_consistent(
            hard, sepa, details=True)
        
        # select the bins with target binary parameters
        # I could update this to select out my single source bins
        sel_mtot = (mtot_range[0] < sam.mtot) & (sam.mtot <= mtot_range[1])
        sel_himt = (mtot_hirng[0] < sam.mtot) & (sam.mtot <= mtot_range[1])
        sel_mrat = (mrat_range[0] < sam.mrat) & (sam.mrat <= mrat_range[1])
        sel_redz = (redz_range[0] < sam.redz) & (sam.redz <= redz_range[1])
        sel = (
            sel_mtot[:, np.newaxis, np.newaxis] *
            sel_mrat[np.newaxis, :, np.newaxis] * 
            sel_redz[np.newaxis, np.newaxis, :]
        )

        sel_high = (
            sel_himt[:, np.newaxis, np.newaxis] *
            sel_mrat[np.newaxis, :, np.newaxis] * 
            sel_redz[np.newaxis, np.newaxis, :]
        )

        tau = _details['tau'][sel].T
        tau_high = _details['tau'][sel_high].T
        _taus.append(tau)
        _taus_high.append(tau_high)

    inner_hcss.append(_hcss)
    inner_hcbg.append(_hcbg)
    inner_taus.append(_taus)
    inner_taus_high.append(_taus_high)
        